In [410]:
!pip install spotipy streamlit tensorflow pandas numpy scikit-learn requests musicbrainzngs

In [374]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pickle
import os
import random
import time
import copy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import math
import matplotlib.pyplot as plt
import streamlit as st #sa speram ca apuc sa-i fac si ui
import musicbrainzngs #pt ca spotipy nu mai ofera song audio features

In [386]:
#CLIENT_ID=os.environ.get('CLIENT_ID')
CLIENT_ID='c2683c1b0a1f4086974e2fe4f8abb9a1'
#CLIENT_SECRET=os.environ.get('CLIENT_SECRET')
CLIENT_SECRET='3dfb3754823542f4b2cd529df78cd5d8'
#REDIRECT_URI = 'https://developer.spotify.com/'  
REDIRECT_URI='https://developer.spotify.com/'
#REDIRECT_URI='http://127.0.0.1:8888/callback'
#REDIRECT_URI = 'https://127.0.0.1:5000/callback'# spotify cere secure, nu merge http asa ca am facut asta cu mkcert si flask disponibil pana in 8 sept

In [388]:
#print(CLIENT_ID, CLIENT_SECRET, REDIRECT_URI)

In [390]:
sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET,
        redirect_uri=REDIRECT_URI,
        scope='user-read-private user-read-email user-read-playback-state user-modify-playback-state user-read-currently-playing user-library-modify user-library-read user-top-read playlist-read-private playlist-modify-public playlist-modify-private playlist-read-collaborative user-follow-read user-follow-modify user-read-recently-played user-read-private'
        #scope='user-library-read user-top-read playlist-read-private playlist-read-collaborative' #de modificat in functie de nevoie
    )
)

In [392]:
st.set_page_config(page_title='TINDIFY', page_icon=':musical_note:')
st.title('Tinder for music tracks')
st.write('Match with new songs that match your vibe! Swipe right to add a track to your favorites, up to add it to a special playlist as well as to your favorites, and left if it is not quite your type')

In [394]:
top_tracks_long=sp.current_user_top_tracks(limit=50, time_range='long_term')
top_tracks_medium=sp.current_user_top_tracks(limit=50, time_range='medium_term')
top_tracks_short=sp.current_user_top_tracks(limit=50, time_range='short_term')

Enter the URL you were redirected to:  https://developer.spotify.com/?code=AQBfocuxZZ0zm0yVLzwa70CO8VgqmE2Uxz5FVVe4Y_tDs8QFJSgHO0F5uIKltwy5Xct_98FgfT0DChpRF8GSSigKHzY4S4dz_PCeXCdIC7oT0yXTLSBEINd3etbIhmFYWzo_uqepcJYsm9cCMzJLlBNPYjFlGx-2WSK_cynaBPV-KUMs4pr3zjHG15pfy_AqI2VZQzRj-Yg3KI0zY4yTU-jdxG4fw19kU6iriE8BuVGt0A78XF6-6JinuWFPs_xXe3bZpeY2EusEGoknr0yUNMMJltmWhsut6ZH58oxP4v6Bdkb-QVldqZBydDW4JlJnWDkmNRAx7Q8j1MpRYCdp-hwAhEs1o0DJbYpRUVwdmXbjHsfUxDYe-fdhgwAAisp4EqBGRavkd3X2nzRJpyq7dj0LAflmGJm_6-CkQ8KLPb6FrJ5dz1YgofySR_HbwS_ucGHGotfSxmy-tlK2BguB0isFKDedHqK-ahN3XiZ1UDfC9ymz5DNCztFxgTPX5mD861_ZaqcD8NbnrLwgiyuYGm2BCrJmtnWc8HjyQUQkQBrDeEAXOvUEsO3W2ngMSZfG088xY6A1GyehO6x9Vewmd7cmDWBolk-ae5B8v6qA69-UhgYm8t4


In [414]:
#top_tracks = list(set([track for track in top_tracks_short].extend([track for track in top_tracks_medium].extend([track for track in top_tracks_long])))) #sa le luam in ordine, dar sa eliminam duplicatele
#top_tracks = list(set([top_tracks_short.extend(top_tracks_medium)].extend(top_tracks_long)))
#nu merge ca aparent e dictionar nu lista


#all_tracks = top_tracks_long['items'] + top_tracks_medium['items'] + top_tracks_short['items']

# seen = set()
# unique_items = []
# for track in all_tracks:
#     tid = track['id']
#     if tid not in seen:
#         unique_items.append(track)  # obiectul full
#         seen.add(tid)

# top_tracks = {
#     'items': unique_items,
#     'total': len(unique_items),
#     'limit': len(unique_items),
#     'offset': 0
# }

#from collections import defaultdict

# merged = defaultdict(set)

# def to_iterable(val):
#     if isinstance(val, (list, set, tuple)):
#         return val
#     return [val]

# for k, v in top_tracks_short.items():
#     merged[k].update(to_iterable(v))
    
# for k, v in top_tracks_medium.items():
#     merged[k].update(to_iterable(v))

# for k, v in top_tracks_long.items():
#     merged[k].update(to_iterable(v))

# top_tracks = {k: sorted(v) for k, v in merged.items()}

merged_items = {}

for track in top_tracks_short.get('items', []):
    merged_items[track['id']] = track

for track in top_tracks_medium.get('items', []):
    merged_items[track['id']] = track

for track in top_tracks_long.get('items', []):
    merged_items[track['id']] = track

top_tracks = {
    'items': list(merged_items.values()),
    'total': len(merged_items)
}

# print(top_tracks_short['total'])
#print(len(top_tracks['items']))

track_ids=[track['name'] for track in top_tracks['items']]

print(track_ids)

track_names=[track['name'] for track in top_tracks['items']]
# print(len(track_names))

['B.M.V.', 'Go Gettas', 'Brasileira - From “TEAMBUILDING” The Movie', 'Paranoid - 2012 - Remaster', 'The Pretender', 'Hand Of Blood', 'Raining Blood', 'One Last Breath', 'Higher', 'Sad But True - Remastered 2021', '7 Words', 'Endorama', 'Voices In My Head', 'Chop Suey!', 'MIDDLE CHILD', 'Treaba Ei', 'Master Of Puppets', 'Freedom', 'Strength Beyond Strength', 'Creep', 'Godzilla (feat. Juice WRLD)', 'No More Tears', 'How You Remind Me', 'Lying from You', 'Symphony Of Destruction - Remastered 2012', 'I Hate Everything About You', 'Creeping Death - Creeping Death EP Version / Remastered', 'This Means War', 'Raping Your Mind', 'Brimstone', 'Mea Culpa (Ah! Ça ira!)', 'Eye of the Beholder', 'Floare de Fier', '808s and Goth Bitches', 'Engine No. 9', 'Love Is a Bitch', 'Supreme Ruler', 'Nuketown (feat. Juice WRLD)', 'To the Hellfire', 'Hit The Floor', 'Toxicity', 'Heart-Shaped Box', 'Agent Orange', 'BETO DANCE', 'Bleed', 'Closer', 'Over the Wall', 'Not Like Us', 'Unholy Confessions', 'One', 'Se

In [412]:
print(sp.auth_manager.get_access_token(as_dict=False))
#sp.audio_features(["6y2DHyCYf6azhUfXmnuH6w"])

BQCOCZc3kqOWG-mQOy9Mn1qL0hLdz13i_7wKmAq3BvyUdKe5bgngTuEetJ2F644Y8mONzfWkJYaFfSgFl-uwBl7-iPd2Ug7788r_s3KLL_WNpTStyQQKZq6JwZvrpENUmTcbnsZ3y3Ls-xZDqIAyr0qMOB02EkTJAlusnJ4u0m1OIogST4wp32eMiCtpxi7GYNwN217FP_Q472tmFoJzPoLhQhtQGimF0g800uu1C27O5EIH4YFeRDBbIPzreXhYxtJS39LI9UOZNP-DpvxNb3c7cpJWmDs1jylGJgMuPZGScXEcVOAQVAMogN_AlOlGT4MojGq44mveY6VjVf7XIRKROQFupCbnwGT3KFE7p1Rebg


In [416]:
#import os
#os.remove(".cache")  # sau .cache-CLIENT_ID dacă e numit așaz

In [420]:
#audio_features = sp.audio_features(track_ids) #spotipy nu mai pune asta la dispozitie

In [422]:
# df = pd.DataFrame(audio_features)
# df['track_name'] = [track['name'] for track in top_tracks['items']]
# df = df[['track_name','danceability','energy','valence','tempo','liveness','instrumentalness','speechiness','acousticness']]
# df.set_index('track_name',inplace=T)

In [424]:
#TEMPORAR, PT VERIFICARE
# st.subheader('barchart taste dataframe:')
# st.bar_chart(df,height=500)

In [ ]:
musicbrainzngs.set_useragent(
    "TINDIFY", "1.0", "https://github.com/david-tampea/tindify"
)